# **BASE ECONOMÍA**

En este notebook se realizará la limpieza de tablas descargadas de la página del banco central, entre la información que se tiene encontramos consumo, tasa de desempleo, e indicadores financieros (cuentas bancarias, saldos, etc.).

In [1]:
#========================================================#
#== Definimos las librerias necesarias para esta parte ==#
#========================================================#

import pandas as pd
import seaborn as sns
import pyarrow.parquet as pq
import pyarrow as pa
import os
import glob

In [128]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
directorio = '/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_crudos_1'

## Importación y limpieza de tablas.



---


### TABLA1: Consumo de hogares e IPSFL

* Consumo de hogares: Es el gasto total realizado por las familias en bienes y
servicios para satisfacer sus necesidades. Incluye cosas como alimentación, vivienda, transporte, educación, salud, etc.
* IPSFL (Instituciones Privadas Sin Fines de Lucro): Son organizaciones no gubernamentales que brindan servicios a los hogares, como fundaciones, asociaciones, y ONGs. Su consumo está relacionado con los servicios que ofrecen (financiados por donaciones, cuotas, etc.).

En conjunto, este indicador mide cuánto se gasta en la región tanto por los hogares como por estas instituciones.

In [88]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_consumo = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Consumo' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      #print(nombre_archivo.split("_", -1)[-1])
      tablas = tablas.T.iloc[2:,[1, 5]].reset_index(drop=True).rename(columns={1:'Periodo_trimestral',5:'Consumo_de_hogares_e_IPSFL'})
      tablas['Region'] = nombre_archivo.split("_", -1)[-1]
      tablas = tablas[['Periodo_trimestral','Consumo_de_hogares_e_IPSFL','Region']]
      df_consumo = pd.concat([df_consumo,tablas],axis=0)  # Concatena cada DataFrame por región

In [89]:
df_consumo['Periodo_trimestral'] = pd.to_datetime(df_consumo['Periodo_trimestral'])
df_consumo

,Periodo_trimestral,Consumo_de_hogares_e_IPSFL,Region
0,2013-03-01,858.607608,Libertador
1,2013-06-01,899.085705,Libertador
2,2013-09-01,897.871339,Libertador
3,2013-12-01,952.105356,Libertador
4,2014-03-01,906.077154,Libertador
...,...,...,...
41,2023-06-01,431.840883,Atacama
42,2023-09-01,428.970689,Atacama
43,2023-12-01,460.335001,Atacama
44,2024-03-01,445.708646,Atacama



Tabla trimestral.

*   Solo se tiene información desde el 2013 hasta el segundo trimestre de el presente año.
*   Información completa para cada región



In [129]:
df_consumo.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_consumo_trimestral.parquet', engine='pyarrow')



---


### TABLA2: Tenencia y saldos en cuentas corrientes y vista (Indicadores financieros)

Número de cuentas corrientes de personas naturales en moneda nacional
- **Descripción:**  
Es la cantidad total de cuentas corrientes pertenecientes a personas naturales (individuos, no empresas) que están denominadas en pesos chilenos.
Indica cuántas personas tienen cuentas corrientes activas en moneda nacional.

Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos)
- **Descripción:**  
Representa el saldo promedio disponible en las cuentas corrientes de personas naturales denominadas en pesos chilenos.
Refleja el comportamiento financiero general de los usuarios en términos de cuánto dinero suelen mantener en sus cuentas.

Número de cuentas corrientes de personas naturales en moneda extranjera
- **Descripción:**  
Es la cantidad total de cuentas corrientes de personas naturales denominadas en una moneda extranjera (por ejemplo, dólares estadounidenses).
Muestra cuántas personas han optado por mantener cuentas en monedas extranjeras, lo que puede estar relacionado con actividades internacionales o protección contra devaluaciones del peso chileno.

Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos)
- **Descripción:**  
Representa el saldo promedio en cuentas corrientes denominadas en monedas extranjeras, convertido a pesos chilenos.
Da una idea de cuánto dinero manejan las personas en cuentas extranjeras, lo que puede ser útil para entender tendencias de dolarización o ahorro en otras monedas.

Saldo de cuentas a la vista por región (en millones de pesos)
- **Descripción:**  
Representa el monto total disponible en cuentas a la vista (similares a cuentas corrientes, pero con menor complejidad) expresado en millones de pesos chilenos.
Es una métrica de liquidez en la región, mostrando cuánto dinero está disponible de manera inmediata en estas cuentas.

In [104]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_Ind_finan = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Ind_finan' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      tablas.columns = tablas.iloc[1]
      tablas = tablas.iloc[2:].reset_index(drop=True)
      tablas = tablas.rename(columns={tablas.columns[-1]:'5.Saldo de cuentas a la vista (en millones de pesos)'})
      tablas['Region'] =  nombre_archivo.split("_", -1)[-1]
      df_Ind_finan = pd.concat([df_Ind_finan,tablas],axis=0)
      #df_consumo = pd.concat([df_consumo,tablas],axis=0)  # Guarda el DataFrame en el diccionario con el nombre del archivo como clave

In [105]:
df_Ind_finan['Periodo'] = pd.to_datetime(df_Ind_finan['Periodo'])
df_Ind_finan['Año_Trimestre'] = df_Ind_finan['Periodo'].dt.to_period('Q')
df_Ind_finan['Periodo_trimestral'] = df_Ind_finan['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
df_Ind_finan.drop(columns={'Periodo','Año_Trimestre'},inplace=True)
df_Ind_finan_trimes = df_Ind_finan.groupby(['Region','Periodo_trimestral']).mean().reset_index()
df_Ind_finan_trimes

1,Region,Periodo_trimestral,1.Número de cuentas corrientes de personas naturales en moneda nacional,2.Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos),3.Número de cuentas corrientes de personas naturales en moneda extranjera,4.Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos),5.Saldo de cuentas a la vista (en millones de pesos)
0,Antofagasta,2013-03-01,96238.333333,2324128.063219,320.0,8372920.426099,53273.287224
1,Antofagasta,2013-06-01,98320.666667,2183286.573678,324.333333,8524981.876501,54060.851445
2,Antofagasta,2013-09-01,100339.0,2097439.229556,330.666667,7060843.187003,52443.110412
3,Antofagasta,2013-12-01,102117.666667,2046252.577891,333.666667,7321857.956338,56020.125436
4,Antofagasta,2014-03-01,103377.0,2133526.007748,334.333333,9050310.192042,56750.909477
...,...,...,...,...,...,...,...
724,Valparaiso,2023-09-01,400536.666667,2222853.341999,30433.333333,1966408.822265,551433.063098
725,Valparaiso,2023-12-01,404575.333333,2190261.081638,32552.666667,1787956.044287,551114.390586
726,Valparaiso,2024-03-01,414492.333333,2251444.034605,36026.333333,1540624.937593,573778.864104
727,Valparaiso,2024-06-01,422535.666667,2248382.459183,39539.0,1416143.537496,580367.59605


Tabla mensual convertida a trimestral.

*   Se tiene información desde el 2013 hasta el tercer trimestre de el presente año.
*   Información completa para cada región, excepto para la region de Ñuble ya que solo tiene info desde el 2018


In [130]:
df_Ind_finan_trimes.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_Ind_finan_trimes.parquet', engine='pyarrow')



---


### TABLA3: Fuerza de Trabajo, Ocupados y Tasa de Desocupación  


Fuerza de trabajo (Miles de personas)  
- **Descripción:**  
  Representa el total de personas económicamente activas, es decir, aquellas que están trabajando o buscan empleo activamente.  

Ocupados (Miles de personas)  
- **Descripción:**  
  Número de personas dentro de la fuerza laboral que están empleadas (que tienen un trabajo).  

Tasa de desocupación (%)  
- **Descripción:**  
  Porcentaje de personas en la fuerza laboral que están desocupadas (buscando empleo activamente pero sin trabajo).  




In [116]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_Ind_des = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Ind_des' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      tablas.columns = tablas.iloc[1]
      tablas = tablas.iloc[2:].reset_index(drop=True)
      tablas['Region'] = nombre_archivo.split('_',-1)[-1]
      df_Ind_des = pd.concat([df_Ind_des,tablas],axis=0)  # Guarda el DataFrame en el diccionario con el nombre del archivo como clave

In [117]:
df_Ind_des

1,Periodo,1.Fuerza de trabajo (Miles de personas),2.Ocupados (Miles de personas),3.Tasa de desocupación (%),Region
0,2010-03-01 00:00:00,105.687017,96.949696,8.267165,Arica
1,2010-04-01 00:00:00,105.020735,96.35523,8.251233,Arica
2,2010-05-01 00:00:00,104.820382,96.723092,7.72492,Arica
3,2010-06-01 00:00:00,104.660821,97.254562,7.076439,Arica
4,2010-07-01 00:00:00,103.144872,97.15242,5.809743,Arica
...,...,...,...,...,...
170,2024-05-01 00:00:00,102.518456,95.27242,7.068031,Magallanes
171,2024-06-01 00:00:00,102.205584,95.948845,6.12172,Magallanes
172,2024-07-01 00:00:00,100.461193,94.553654,5.880419,Magallanes
173,2024-08-01 00:00:00,102.299899,97.14002,5.043875,Magallanes


In [118]:
df_Ind_des['Periodo'] = pd.to_datetime(df_Ind_des['Periodo'])
df_Ind_des = df_Ind_des[df_Ind_des['Periodo']>'2012-12-01']
df_Ind_des['Año_Trimestre'] = df_Ind_des['Periodo'].dt.to_period('Q')
df_Ind_des['Periodo_trimestral'] = df_Ind_des['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
df_Ind_des.drop(columns={'Periodo','Año_Trimestre'},inplace=True)
df_Ind_des_trimes = df_Ind_des.groupby(['Region','Periodo_trimestral']).mean().reset_index()
df_Ind_des_trimes

<ipython-input-118-1c42ead32045>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Ind_des['Año_Trimestre'] = df_Ind_des['Periodo'].dt.to_period('Q')
<ipython-input-118-1c42ead32045>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Ind_des['Periodo_trimestral'] = df_Ind_des['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
<ipython-input-118-1c42ead32045>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

1,Region,Periodo_trimestral,1.Fuerza de trabajo (Miles de personas),2.Ocupados (Miles de personas),3.Tasa de desocupación (%)
0,Antofagasta,2013-03-01,269.113297,253.510501,5.797311
1,Antofagasta,2013-06-01,277.229068,262.261608,5.399044
2,Antofagasta,2013-09-01,273.138637,258.056848,5.521059
3,Antofagasta,2013-12-01,280.224621,262.012615,6.500208
4,Antofagasta,2014-03-01,278.252303,257.621397,7.413592
...,...,...,...,...,...
747,Valparaiso,2023-09-01,994.064092,903.022493,9.157725
748,Valparaiso,2023-12-01,993.825406,910.162555,8.418221
749,Valparaiso,2024-03-01,1001.435912,911.514531,8.979508
750,Valparaiso,2024-06-01,992.174294,911.392222,8.142903


Tabla mensual convertida a trimestral.

*   Se tiene información desde el 2013 hasta el tercer trimestre de el presente año.
*   Información completa para cada región.

In [131]:
df_Ind_des_trimes.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_Ind_des_trimes.parquet', engine='pyarrow')

## Merge de las tablas

In [134]:
path = '/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/'

In [142]:
df_consumo_trimes = pd.read_parquet(f'{path}df_consumo_trimestral.parquet', engine='pyarrow')
df_Ind_finan_trimes = pd.read_parquet(f'{path}df_Ind_finan_trimes.parquet', engine='pyarrow')
df_Ind_des_trimes = pd.read_parquet(f'{path}df_Ind_des_trimes.parquet', engine='pyarrow')

In [144]:
print('df_consumo_trimes:',df_consumo_trimes.shape)
print('df_Ind_finan_trimes:',df_Ind_finan_trimes.shape)
print('df_Ind_des_trimes:',df_Ind_des_trimes.shape)

df_consumo_trimes: (736, 3)
df_Ind_finan_trimes: (729, 7)
df_Ind_des_trimes: (752, 5)


In [148]:
df_economia_final = df_consumo_trimes.merge(df_Ind_finan_trimes, how = 'left', on = ['Periodo_trimestral','Region'])
df_economia_final = df_economia_final.merge(df_Ind_des_trimes, how = 'left', on = ['Periodo_trimestral','Region'])
df_economia_final.shape

(736, 11)

In [151]:
df_economia_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 11 columns):
 #   Column                                                                                        Non-Null Count  Dtype         
---  ------                                                                                        --------------  -----         
 0   Periodo_trimestral                                                                            736 non-null    datetime64[ns]
 1   Consumo_de_hogares_e_IPSFL                                                                    736 non-null    float64       
 2   Region                                                                                        736 non-null    object        
 3   1.Número de cuentas corrientes de personas naturales en moneda nacional                       713 non-null    float64       
 4   2.Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos)    713 non-null    

- Se hizo un merge tomando como referencia el tamaño de la tabla de consumo ya que solo cuenta con información hasta el segundo trimestre de este periodo.
- Se observa que hay datos faltantes, es debido que la región Ñuble no cuenta con esa información en la página del Banco Central.

In [150]:
df_economia_final.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/df_economia_final.parquet', engine='pyarrow')